In [1]:
from Scripts.utils import *  
import pandas as pd
#columns = ['Date','Open','High','Low','Close','Volume']
#index = ['Date']
#data = generateData('Data/EURUSD/EURUSD_D1.csv', columns, index, indicators = True)
columns = ['Date','Open','High','Low','Close','Volume','Yield','PercentageVolume',
           'SMA6','EMA6','WMA6','HMA6','SMA20','EMA20','WMA20','HMA20','SMA50','EMA50','WMA50','HMA50',
           'SMA100','EMA100','WMA100','HMA100','MACD','CCI','Stochastic Oscillator','RSI','ROC','PPO',
           'KST','BOLU','BOLD','BOLM']
data = pd.read_csv('DataReady/EURUSD/EURUSD_D1.csv', names = columns, header = 0)

In [2]:
toRemove = ['Volume', 'Date','High','Low','Open','Close']
df = selectData(data,toRemove)

In [3]:
dfDiff = df.diff().dropna(how='any')
normDf = normalizeData(dfDiff)
images = generateImages(normDf)

In [4]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(images, data['Close'][29:], test_size = 0.2,shuffle = True)
train_X, valid_X, train_Y, valid_Y = train_test_split(train_X, train_Y, test_size = 0.1,shuffle = True)

In [5]:
train_X = np.array([elem.reshape(28*28,1) for elem in train_X])
valid_X = np.array([elem.reshape(28*28,1) for elem in valid_X])
test_X = np.array([elem.reshape(28*28,1) for elem in test_X])

In [6]:
epochs_set = [30,70,200]
batches = [32]
learning_rates = [0.0005,0.001,0.01,0.1]
num_channels = [2,3,4,5,6,7,8]
nb_filters = [25,64,100]

In [7]:
combinations = []
for epochs in epochs_set:
    for batch_size in batches:
        for learning_rate in learning_rates:
            for num_channel in num_channels:
                for nb_filter in nb_filters:
                    combinations.append([epochs, batch_size,learning_rate, num_channel, nb_filter])

In [8]:
print(combinations[167])

[70, 32, 0.1, 8, 100]


In [ ]:
from Scripts.TCN import *

mse_min = 0.0024803249657470256
corr_max = 0.9347684037123097

best_factor = np.sqrt(mse_min) * (1-corr_max)
for comb in combinations[167:]:
    best = False
    epochs, batch_size,learning_rate, num_channel, nb_filter = comb
    predictedPrices, mse, corr = run(train_X, valid_X, test_X, train_Y, valid_Y,test_Y,
                                    epochs, batch_size,learning_rate, num_channel, nb_filter, mse_min, corr_max)
    print("{},{}".format(mse,corr))
    if corr == corr and corr>0:
        factor = np.sqrt(mse)*(1-corr)
        if factor < best_factor:
            mse_min = mse
            corr_max = corr
            best_factor = factor
            best = True
            print("BEST")
    f = open('Results/resultsTCNdiff.txt','a+')
    f.write("{},{},{},{},3,{},{},{}, {}\n".format(epochs,batch_size,num_channel,nb_filter,learning_rate,mse, corr, best))
    f.close()